In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

train = pd.read_csv('train.csv')

x= train.drop('target',axis=1)
y= train['target']

x.head()



,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,0,0,0,0,T,Y,Green,Triangle,Snake,Finland,...,c389000ab,2f4cb3d51,2,Grandmaster,Cold,h,D,kr,2,2
1,1,0,1,0,T,Y,Green,Trapezoid,Hamster,Russia,...,4cd920251,f83c56c21,1,Grandmaster,Hot,a,A,bF,7,8
2,2,0,0,0,F,Y,Blue,Trapezoid,Lion,Russia,...,de9c9f684,ae6800dd0,1,Expert,Lava Hot,h,R,Jc,7,2
3,3,0,1,0,F,Y,Red,Trapezoid,Snake,Canada,...,4ade6ab69,8270f0d71,1,Grandmaster,Boiling Hot,i,D,kW,2,1
4,4,0,0,0,F,N,Red,Trapezoid,Lion,Canada,...,cb43ab175,b164b72a7,1,Grandmaster,Freezing,a,R,qP,7,8


In [2]:
y.value_counts()

0    208236
1     91764
Name: target, dtype: int64

In [3]:
x.sample(frac=1).reset_index(drop=True)

,id,bin_0,bin_1,bin_2,bin_3,bin_4,nom_0,nom_1,nom_2,nom_3,...,nom_8,nom_9,ord_0,ord_1,ord_2,ord_3,ord_4,ord_5,day,month
0,169713,0,0,1,F,N,Red,Polygon,Snake,Russia,...,407ee1590,5966d4e2f,1,Novice,Lava Hot,j,Z,mm,7,9
1,159735,1,0,0,T,Y,Red,Trapezoid,Axolotl,India,...,dd3966979,24708272a,1,Grandmaster,Hot,h,Q,GD,1,5
2,163861,0,0,1,T,Y,Red,Trapezoid,Axolotl,Costa Rica,...,7fc4ec511,cc7de29ab,2,Contributor,Lava Hot,j,S,MC,3,3
3,42985,0,0,1,T,N,Green,Polygon,Hamster,Russia,...,6d2a862e0,626099be6,1,Grandmaster,Cold,a,P,LE,2,4
4,182191,0,0,0,F,Y,Red,Circle,Lion,India,...,a05cfe94b,4cb766b33,3,Novice,Lava Hot,l,L,kW,1,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
299995,285275,1,0,1,F,N,Blue,Square,Lion,Russia,...,17ba84114,7c8cf939d,1,Novice,Lava Hot,l,R,cW,2,7
299996,249194,0,1,0,T,Y,Red,Trapezoid,Lion,Russia,...,efb91e4dd,54ef03d57,1,Novice,Freezing,o,X,PA,5,8
299997,101153,0,0,0,T,Y,Blue,Trapezoid,Lion,Finland,...,1086ce405,95eb51be1,1,Novice,Lava Hot,c,S,lF,3,12
299998,83573,1,0,1,F,Y,Red,Triangle,Dog,Russia,...,63572f4db,3cc46448a,1,Novice,Lava Hot,h,H,NV,4,9


In [10]:
from sklearn import model_selection
class crossvalidation:
    def __init__(self,dataframe,target_cols,shuffle,multi_delimiter=',',n_folds=5,problem_type='binary_classification'):
        self.dataframe = dataframe
        self.target_cols = target_cols
        self.num_targets = len(target_cols)
        self.shuffle = shuffle
        self.n_folds = n_folds
        self.problem_type = problem_type
        self.multi_delimiter = multi_delimiter
        self.dataframe['kfold']=-1
        if self.shuffle is True:
                self.dataframe = self.dataframe.sample(frac=1).reset_index(drop=True)

    
    def split(self):
        if self.problem_type in  ["binary_classification", "multiclass_classification"]:
            if self.num_targets!=1:
                raise Exception('invalid number of targets')
            target = self.target_cols[0]
            unique_values = self.dataframe[target].nunique()   
            if unique_values == 1:
                raise Exception('Only one unique value')     
            if unique_values>1:
                kf = model_selection.StratifiedKFold(n_splits=self.n_folds)
                for fold, (train_index,val_index) in enumerate(kf.split(X=self.dataframe,y=self.dataframe[target])):
                    self.dataframe.loc[val_index,'kfold']= fold
                    
        ### For Single target column Regression so use simple Kfold instead of startified one
        elif self.problem_type in ('Single_col_regression','multi_col_regression'):
            if self.num_targets != 1 and self.problem_type =='Single_col_regression':
                raise Exception('invalid number of targets')
            if self.num_targets < 2 and self.problem_type =='multi_col_regression':
                raise Exception('invalid number of targets')  
            kf = model_selection.KFold(n_splits=self.n_folds)
            for fold, (train_index,val_index) in enumerate(X=kf.split(self.dataframe)):
                self.dataframe.loc[val_index,'kfold']= fold
        
        ## most of the time used for the TIME SERIES data and we do not shuffle the data i.e. shuffle=False
        ## almost similar to train_test_split but do not shuffle the data
        elif self.problem_type.startswith('holdout_'):
            percentage = int(self.problem_type.split('_')[1])
            holdout_index = len(self.dataframe)*percentage/100
            self.dataframe.loc[:len(self.dataframe)-holdout_index,'kfold'] = 0
            self.dataframe.loc[len(self.dataframe)-holdout_index:,'kfold'] = 1

        ## multilabel cross validation like target has insatnces as (1,2) , (3,4,5) or many lables like that 
        elif self.problem_type == 'multilabel_classification':
            if self.num_targets != 1:
                raise Exception('invalid number of targets')
            targets = self.dataframe[self.target_cols[0]].apply(lambda x:len(str(x).split(self.multi_delimiter)))
            if sum(targets)==len(self.dataframe):
#                 print('not this type of problem')
                raise Exception('not this type of problem')
            kf = model_selection.StratifiedKFold(n_splits=self.n_folds)
            for fold, (train_index,val_index) in enumerate(kf.split(X=self.dataframe,y=targets)):
                self.dataframe.loc[val_index,'kfold']= fold
        
        else: 
            raise Exception('problem type not understood')
                    
        return self.dataframe
        
     

In [11]:
if __name__=="__main__":
    df= pd.read_csv('train.csv')
    cv = crossvalidation(df,target_cols=['target'],shuffle=True)
    df_split = cv.split()
    print(df_split.kfold.value_counts())


4    60000
3    60000
2    60000
1    60000
0    60000
Name: kfold, dtype: int64


In [12]:
if __name__=="__main__":
    df= pd.read_csv('train.csv')
    cv = crossvalidation(df,target_cols=['target'],shuffle = False,problem_type='holdout_10')
    df_split = cv.split()
    print(df_split.kfold.value_counts())


0    270000
1     30000
Name: kfold, dtype: int64


In [13]:
if __name__ == "__main__":
    df= pd.read_csv('train.csv') ## data not for this type of problem
    cv = crossvalidation(df,target_cols=['target'],shuffle = True,multi_delimiter=' ',problem_type='multilabel_classification')
    df_split = cv.split()
    print(df_split.kfold.value_counts())


Exception: not this type of problem